### Load Models

In [16]:
from tensorflow.python.keras.models import load_model

Encoder and decoder model

In [17]:
encoder_model = load_model('models/seq2seq_encoder_eng_hin.hd5')

In [18]:
decoder_model = load_model('models/seq2seq_decoder_eng_hin.hd5')

### Load tokenizers

In [19]:
import pickle

In [20]:
encoder_t = pickle.load(open('models/encoder_tokenizer_eng','rb'))

In [21]:
decoder_t = pickle.load(open('models/decoder_tokenizer_hin','rb'))

Define Configuration parameters

In [22]:
max_encoder_seq_length = 22 #From the training

In [23]:
max_decoder_seq_length = 27 #From the training

In [24]:
int_to_word_decoder = dict((i,c) for c, i in decoder_t.word_index.items())

### Function to generate Padded sequences for Input string

In [25]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [26]:
def encode_input(input_str):
    
    #Convert words to indexes
    encoder_seq = encoder_t.texts_to_sequences([input_str])
    
    #Pad sequences
    encoder_input_data = pad_sequences(encoder_seq, maxlen=max_encoder_seq_length, padding='post')
    
    return encoder_input_data    

### Prediction Function

In [27]:
import numpy as np

In [28]:
def decode_sentence(input_str):
    
    #Convert input string to padded sequence
    input_seq = encode_input(input_str)
    
    #Get the encoder state values
    decoder_initial_states_value = encoder_model.predict(input_seq)
    
    #Build a sequence with '<start>' - starting sequence for Decoder
    target_seq = np.zeros((1,1))    
    target_seq[0][0] = decoder_t.word_index['<start>']
    
    #flag to check if prediction should be stopped
    stop_loop = False
    
    #Initialize predicted sentence
    predicted_sentence = ''
    
    #start the loop
    while not stop_loop:
        
        predicted_outputs, h, c = decoder_model.predict([target_seq] + 
                                                        decoder_initial_states_value)
        
        #Get the predicted output with highest probability
        predicted_output = np.argmax(predicted_outputs[0,-1,:])
        
        #Get the predicted word from predicter integer
        predicted_word = int_to_word_decoder[predicted_output]
        
        #Check if prediction should stop
        if(predicted_word == '<end>' or len(predicted_sentence) > max_decoder_seq_length):
            
            stop_loop = True
            continue
                    
        #Updated predicted sentence
        if (len(predicted_sentence) == 0):
            predicted_sentence = predicted_word
        else:
            predicted_sentence = predicted_sentence + ' ' + predicted_word
            
        #Update target_seq to be the predicted word index
        target_seq[0][0] = predicted_output
        
        #Update initial states value forr decoder
        decoder_initial_states_value = [h,c]
        
    
    return predicted_sentence

In [ ]:
if __name__== '__main__':
    print(decode_sentence('How are you?'))

In [31]:
decode_sentence("I'm starving.")

'मैं भूख से मरा जा रहा हूँ।'